In [1]:
import sys
print(sys.version)

3.7.17 (default, Nov  2 2024, 13:11:17) 
[Clang 15.0.0 (clang-1500.3.9.4)]


In [3]:
import os
os.environ['SPARK_HOME'] = "/Users/yoosj/Desktop/datapipe/venv1/Analisis/spark-2.4.8-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

In [5]:
import pyspark
sc = pyspark.SparkContext(appName="SparkExample")

24/11/02 15:02:41 WARN Utils: Your hostname, yuseungjaeui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.12 instead (on interface en0)
24/11/02 15:02:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/11/02 15:02:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/02 15:02:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Transaction

## map

In [6]:
wordslist = [['cat', 'cat'], ['ele'],['cat', 'ele']]
rdd = sc.parallelize(wordslist)
output = rdd.map(lambda line: [x for x in line]).collect()

print(output)

[['cat', 'cat'], ['ele'], ['cat', 'ele']]


## flatMap

In [8]:
wordslist = [['cat', 'cat'], ['ele'],['cat', 'ele']]
rdd = sc.parallelize(wordslist)
output = rdd.flatMap(lambda line: [x for x in line]).collect()

print(output)

['cat', 'cat', 'ele', 'cat', 'ele']


## filter

In [13]:
def remove_cat(animal):
    return(animal != 'cat')

wordslist = [['cat', 'cat'], ['ele'],['cat', 'ele']]
rdd = sc.parallelize(wordslist)
rdd.flatMap(lambda line: [x for x in line]) \
    .filter(remove_cat).collect()

['ele', 'ele']

## Aggregation

In [18]:
def remove_cat(animal):
    return(animal != 'cat')

wordslist = [['cat', 'cat'], ['ele'],['cat', 'ele']]
rdd = sc.parallelize(wordslist)
rdd.flatMap(lambda line: [x for x in line]) \
    .filter(remove_cat) \
    .map(lambda ele: (ele,1)) \
    .reduceByKey(lambda x,y : x + y) \
    .collect()

[('ele', 2)]

# Assignment

## wordcount

In [20]:
wordsList = ['cat', 'ele', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList)

output = wordsRDD.map(lambda x: (x,1)) \
.reduceByKey(lambda x,y: x+y)

output.collect()

[('ele', 1), ('rat', 2), ('cat', 2)]

## 2 

mapping

In [26]:
users = [(0, "Lee"), (1, "Jang"), (2, "Yoo"), (3, "Park"), (4, "Kim")]
subs = [(0, "soccer"), (2, "bread")]
usersRDD = sc.parallelize(users)
subsRDD = sc.parallelize(subs)

mergedRDD = usersRDD.rightOuterJoin(subsRDD)
mergedRDD.map(lambda value : value[1][0] + ' loves ' + value[1][1]).collect()

['Lee loves soccer', 'Yoo loves bread']

## 3

In [34]:
import json

def lovePandas(line):
    k = 'lovePandas'
    if k in line and line[k]:
        return True
    return False

path = "./data/"
input_pile = sc.textFile(path + "pandas.json")

# Json => Python dictionary
pandasJson = input_pile.map(lambda x: json.loads(x))

# Python dictionary => Json
output = pandasJson.filter(lovePandas).map(lambda x: json.dumps(x))
output.collect()

['{"name": "Sparky The Bear", "lovePandas": true}',
 '{"name": "Sparky The Bear", "lovePandas": true, "knows": {"friends": ["holden"]}}']